## DIA 131: Implementar Paginación en el Historial de Reportes

Hoy vas a mejorar la visualización del historial de reportes implementando paginación, para:

Evitar cargar demasiados registros a la vez

Mejorar el rendimiento

Hacer la navegación más ordenada

🧩 1. Modificar la ruta historial_reportes para paginar
Utiliza paginate() de SQLAlchemy:

python
Copiar
Editar
@app.route('/historial')
@login_required
def historial_reportes():
    page = request.args.get('page', 1, type=int)
    query = request.args.get('q', '').strip()
    categoria_filtro = request.args.get('categoria', '').strip()

    # Obtener categorías únicas
    categorias = db.session.query(Reporte.categoria).filter_by(user_id=current_user.id).distinct().all()
    categorias = [c[0] for c in categorias if c[0]]

    # Construir query base
    reportes_query = Reporte.query.filter_by(user_id=current_user.id)

    if query:
        reportes_query = reportes_query.filter(
            db.or_(
                Reporte.comentario.ilike(f'%{query}%'),
                Reporte.categoria.ilike(f'%{query}%'),
                Reporte.url.ilike(f'%{query}%')
            )
        )

    if categoria_filtro:
        reportes_query = reportes_query.filter(Reporte.categoria.ilike(f'%{categoria_filtro}%'))

    reportes_pag = reportes_query.order_by(Reporte.fecha_generado.desc()).paginate(page=page, per_page=10)

    return render_template(
        "historial_reportes.html",
        reportes=reportes_pag.items,
        pagination=reportes_pag,
        categorias=categorias,
        query=query,
        categoria_filtro=categoria_filtro
    )
🧩 2. Mostrar la navegación de páginas en historial_reportes.html
Al final de tu tabla, agrega:

html
Copiar
Editar
<div style="text-align: center; margin-top: 20px;">
  {% if pagination.has_prev %}
    <a href="{{ url_for('historial_reportes', page=pagination.prev_num, q=query, categoria=categoria_filtro) }}">← Anterior</a>
  {% endif %}

  Página {{ pagination.page }} de {{ pagination.pages }}

  {% if pagination.has_next %}
    <a href="{{ url_for('historial_reportes', page=pagination.next_num, q=query, categoria=categoria_filtro) }}">Siguiente →</a>
  {% endif %}
</div>